In [1]:
import os
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Add, ZeroPadding2D, UpSampling2D, Concatenate, MaxPooling2D, LeakyReLU, BatchNormalization, Input, Lambda
from tensorflow.keras.regularizers import l2

from PIL import Image
from functools import reduce

In [2]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
def compose(*funcs):
    """Compose arbitrarily many functions, evaluated left to right.

    Reference: https://mathieularose.com/function-composition-in-python/
    """
    # return lambda x: reduce(lambda v, f: f(v), funcs, x)
    if funcs:
        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)
    else:
        raise ValueError('Composition of empty sequence not supported.')
        
#依次执行参数对应的函数操作 简化代码

In [4]:
def DarknetConv2D(*args, **kwargs):
    """Wrapper to set Darknet parameters for Convolution2D."""
    darknet_conv_kwargs = {'kernel_regularizer': l2(5e-4)} #正则化
    darknet_conv_kwargs['padding'] = 'valid' if kwargs.get('strides') == (2, 2) else 'same' #加边
    darknet_conv_kwargs.update(kwargs) #更新参数
    return Conv2D(*args, **darknet_conv_kwargs)


def DarknetConv2D_BN_Leaky(*args, **kwargs):
    """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""
    no_bias_kwargs = {'use_bias': False}
    no_bias_kwargs.update(kwargs)
    return compose(
        DarknetConv2D(*args, **no_bias_kwargs),
        BatchNormalization(),
        LeakyReLU(alpha=0.1))
#卷积+BN+激活

def resblock_body(x, num_filters, num_blocks):
    '''A series of resblocks starting with a downsampling Convolution2D'''
    # Darknet uses left and top padding instead of 'same' mode
    x = ZeroPadding2D(((1, 0), (1, 0)))(x)
    x = DarknetConv2D_BN_Leaky(num_filters, (3, 3), strides=(2, 2))(x)
    for i in range(num_blocks):
        y = compose(
            DarknetConv2D_BN_Leaky(num_filters // 2, (1, 1)),
            DarknetConv2D_BN_Leaky(num_filters, (3, 3)))(x)
        x = Add()([x, y])
    return x
#加边+卷积模块+拼接

def darknet_body(x):
    '''Darknent body having 52 Convolution2D layers'''
    x = DarknetConv2D_BN_Leaky(32, (3, 3))(x)
    x = resblock_body(x, 64, 1)
    x = resblock_body(x, 128, 2)
    x = resblock_body(x, 256, 8)
    x = resblock_body(x, 512, 8)
    x = resblock_body(x, 1024, 4)
    return x

#卷积部分

def make_last_layers(x, num_filters, out_filters):
    '''6 Conv2D_BN_Leaky layers followed by a Conv2D_linear layer'''
    x = compose(
        DarknetConv2D_BN_Leaky(num_filters, (1, 1)),
        DarknetConv2D_BN_Leaky(num_filters * 2, (3, 3)),
        DarknetConv2D_BN_Leaky(num_filters, (1, 1)),
        DarknetConv2D_BN_Leaky(num_filters * 2, (3, 3)),
        DarknetConv2D_BN_Leaky(num_filters, (1, 1)))(x)
    y = compose(
        DarknetConv2D_BN_Leaky(num_filters * 2, (3, 3)),
        DarknetConv2D(out_filters, (1, 1)))(x)
    return x, y

#输出部分

In [5]:
def yolo_body(inputs, num_anchors, num_classes):
    """创建 YOLOv3 模型的主体
    - inputs: 模型输入
    - num_anchors: 每个尺度的锚点数量
    - num_classes: 类别数量
    """

    # 创建 Darknet 主体网络
    darknet = Model(inputs, darknet_body(inputs))

    # 构建模型的最后几层并获取第一个尺度的输出
    x, y1 = make_last_layers(darknet.output, 512, num_anchors * (num_classes + 5))

    # 上采样并与前一层的输出进行连接
    x = compose(
        DarknetConv2D_BN_Leaky(256, (1, 1)),
        UpSampling2D(2))(x)
    x = Concatenate()([x, darknet.layers[152].output])
    # 构建模型的第二个尺度的输出
    x, y2 = make_last_layers(x, 256, num_anchors * (num_classes + 5))

    # 再次上采样并与前一层的输出进行连接
    x = compose(
        DarknetConv2D_BN_Leaky(128, (1, 1)),
        UpSampling2D(2))(x)
    x = Concatenate()([x, darknet.layers[92].output])
    # 构建模型的第三个尺度的输出
    x, y3 = make_last_layers(x, 128, num_anchors * (num_classes + 5))

    # 返回包含三个尺度输出的模型
    return Model(inputs, [y1, y2, y3])

In [6]:
def yolo_head(feats, anchors, num_classes, input_shape, calc_loss=False):
    """Convert final layer features to bounding box parameters.
    - feats: 特征图，模型最后一层的输出
    - anchors: 锚点数组，定义了一系列预设的边界框尺寸
    - num_classes: 目标类别总数
    - input_shape: 输入尺寸
    - calc_loss: 是否用于损失计算
    """

    # 计算锚点数量
    num_anchors = len(anchors)
    # 将锚点数据转换为 Keras 张量
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    # 获取特征图的高度和宽度
    grid_shape = K.shape(feats)[1:3]  # height, width
    # 生成网格坐标
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),[1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),[grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, K.dtype(feats))

    # 重塑特征图以适应边界框的形状
    feats = K.reshape(feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # 调整预测结果到每个空间网格点和锚点尺寸
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    # 如果用于损失计算，返回网格和各种调整后的预测值
    if calc_loss == True:
        return grid, feats, box_xy, box_wh
    # 否则，返回边界框的坐标、尺寸、置信度和类别概率
    return box_xy, box_wh, box_confidence, box_class_probs

In [7]:
def box_iou(b1, b2):
    """
    Return IoU tensor
    Parameters:
    - b1: tensor, shape=(i1,...,iN, 4), xywh  # 预测边界框
    - b2: tensor, shape=(j, 4), xywh          # 真实边界框
    Returns:
    - iou: tensor, shape=(i1,...,iN, j)       # IoU值
    """

    # 对第一组边界框进行扩展，以便进行广播运算
    b1 = K.expand_dims(b1, -2)
    # 提取中心点坐标和尺寸
    b1_xy = b1[..., :2]
    b1_wh = b1[..., 2:4]
    # 计算边界框的左上角和右下角坐标
    b1_wh_half = b1_wh / 2.
    b1_mins = b1_xy - b1_wh_half
    b1_maxes = b1_xy + b1_wh_half

    # 对第二组边界框进行类似处理
    b2 = K.expand_dims(b2, 0)
    b2_xy = b2[..., :2]
    b2_wh = b2[..., 2:4]
    b2_wh_half = b2_wh / 2.
    b2_mins = b2_xy - b2_wh_half
    b2_maxes = b2_xy + b2_wh_half

    # 计算两组边界框交集的坐标
    intersect_mins = K.maximum(b1_mins, b2_mins)
    intersect_maxes = K.minimum(b1_maxes, b2_maxes)
    intersect_wh = K.maximum(intersect_maxes - intersect_mins, 0.)
    # 计算交集区域的面积
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    # 计算每组边界框的面积
    b1_area = b1_wh[..., 0] * b1_wh[..., 1]
    b2_area = b2_wh[..., 0] * b2_wh[..., 1]
    # 计算IoU值
    iou = intersect_area / (b1_area + b2_area - intersect_area)

    return iou

In [9]:
def yolo_loss(args, anchors, num_classes, ignore_thresh=.5, print_loss=False):
    """
    计算 YOLO 损失
    Parameters:
    - args: 输入参数，包括 yolo_outputs 和 y_true
    - anchors: 锚点尺寸
    - num_classes: 类别数
    - ignore_thresh: 忽略阈值，用于过滤某些预测框
    - print_loss: 是否打印损失信息
    Returns:
    - loss: 计算得到的损失值
    """

    # 计算层数
    num_layers = len(anchors) // 3  # 默认设置
    # 获取 YOLO 输出和真实标签
    yolo_outputs = args[:num_layers]
    y_true = args[num_layers:]
    # 定义每层使用的锚点
    anchor_mask = [[6, 7, 8], [3, 4, 5], [0, 1, 2]] if num_layers == 3 else [[3, 4, 5], [1, 2, 3]]

    # 计算输入尺寸和网格尺寸
    input_shape = K.cast(K.shape(yolo_outputs[0])[1:3] * 32, K.dtype(y_true[0]))
    grid_shapes = [K.cast(K.shape(yolo_outputs[l])[1:3], K.dtype(y_true[0])) for l in range(num_layers)]

    # 初始化损失
    loss = 0
    # 获取批次大小
    m = K.shape(yolo_outputs[0])[0]  # batch size, tensor
    # 批次大小转换为浮点型
    mf = K.cast(m, K.dtype(yolo_outputs[0]))

    # 遍历每层
    for l in range(num_layers):
        # 提取对象掩码和类别概率
        object_mask = y_true[l][..., 4:5]
        true_class_probs = y_true[l][..., 5:]

        # 调用 yolo_head 获取预测框
        grid, raw_pred, pred_xy, pred_wh = yolo_head(yolo_outputs[l],
                                                     anchors[anchor_mask[l]], num_classes, input_shape, calc_loss=True)
        # 合并预测框坐标
        pred_box = K.concatenate([pred_xy, pred_wh])

        # 处理真实框，计算损失
        raw_true_xy = y_true[l][..., :2] * grid_shapes[l][::-1] - grid
        raw_true_wh = K.log(y_true[l][..., 2:4] / anchors[anchor_mask[l]] * input_shape[::-1])
        raw_true_wh = K.switch(object_mask, raw_true_wh, K.zeros_like(raw_true_wh))  # avoid log(0)=-inf
        box_loss_scale = 2 - y_true[l][..., 2:3] * y_true[l][..., 3:4]

        # 寻找忽略掩码
        ignore_mask = tf.TensorArray(K.dtype(y_true[0]), size=1, dynamic_size=True)
        object_mask_bool = K.cast(object_mask, 'bool')

        def loop_body(b, ignore_mask):
            true_box = tf.boolean_mask(y_true[l][b, ..., 0:4], object_mask_bool[b, ..., 0])
            iou = box_iou(pred_box[b], true_box)
            best_iou = K.max(iou, axis=-1)
            ignore_mask = ignore_mask.write(b, K.cast(best_iou < ignore_thresh, K.dtype(true_box)))
            return b + 1, ignore_mask

        _, ignore_mask = tf.while_loop(lambda b, *args: b < m, loop_body, [0, ignore_mask])
        ignore_mask = ignore_mask.stack()
        ignore_mask = K.expand_dims(ignore_mask, -1)

        # 计算各部分损失
#         xy_loss = object_mask * box_loss_scale * K.binary_crossentropy(raw_true_xy, raw_pred[..., 0:2],
#                                                                        from_logits=True)
        xy_loss = object_mask * box_loss_scale * 0.5 * K.square(raw_true_xy - raw_pred[..., 0:2])
        wh_loss = object_mask * box_loss_scale * 0.5 * K.square(raw_true_wh - raw_pred[..., 2:4])
        confidence_loss = object_mask * K.binary_crossentropy(object_mask, raw_pred[..., 4:5], from_logits=True) + \
                          (1 - object_mask) * K.binary_crossentropy(object_mask, raw_pred[..., 4:5],
                                                                    from_logits=True) * ignore_mask
        class_loss = object_mask * K.binary_crossentropy(true_class_probs, raw_pred[..., 5:], from_logits=True)

        # 将每部分损失求和并除以批次大小，实现损失的归一化
        xy_loss = K.sum(xy_loss) / mf
        wh_loss = K.sum(wh_loss) / mf
        confidence_loss = K.sum(confidence_loss) / mf
        class_loss = K.sum(class_loss) / mf
        # 将所有损失项累加得到总损失
        loss += xy_loss + wh_loss + confidence_loss + class_loss
        # 如果需要，打印损失信息
        if print_loss:
            loss = tf.print(loss, [loss, xy_loss, wh_loss, confidence_loss, class_loss, K.sum(ignore_mask)],
                            message='loss: ')
    # 返回计算得到的总损失
    return loss


    '''Return yolo_loss tensor

    Parameters
    ----------
    yolo_outputs: list of tensor, the output of yolo_body or tiny_yolo_body
    y_true: list of array, the output of preprocess_true_boxes
    anchors: array, shape=(N, 2), wh
    num_classes: integer
    ignore_thresh: float, the iou threshold whether to ignore object confidence loss

    Returns
    -------
    loss: tensor, shape=(1,)

    '''

In [10]:
def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2, weights_path=''):
    '''创建用于训练的 YOLOv3 模型
    - input_shape: 输入尺寸
    - anchors: 锚点
    - num_classes: 类别数量
    - load_pretrained: 是否加载预训练权重
    - freeze_body: 冻结层的模式
    - weights_path: 预训练权重的路径
    '''

    # 定义模型的输入层
    image_input = Input(shape=(None, None, 3))
    # 获取输入尺寸的高和宽
    h, w = input_shape
    # 计算锚点数量
    num_anchors = len(anchors)

    # 定义真实标签的输入层
    y_true = [Input(shape=(h // {0: 32, 1: 16, 2: 8}[l], w // {0: 32, 1: 16, 2: 8}[l], num_anchors // 3, num_classes + 5)) for l in range(3)]

    # 创建 YOLOv3 模型主体
    model_body = yolo_body(image_input, num_anchors // 3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    # 如果需要加载预训练权重
    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        # 根据 freeze_body 参数冻结部分层
        if freeze_body in [1, 2]:
            num = (185, len(model_body.layers) - 3)[freeze_body - 1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    # 创建损失函数层
    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
                        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    # 构建完整模型
    model = Model([model_body.input, *y_true], model_loss)

    return model

In [11]:
def get_random_data(annotation_line, input_shape, random=False, max_boxes=20):
    '''random preprocessing for real-time data augmentation
    - annotation_line: 一行标注数据，包含图像路径和边界框信息
    - input_shape: 模型输入尺寸
    - random: 是否应用随机变换
    - max_boxes: 每张图像的最大边界框数量
    '''

    # 分割标注行，获取图像路径和边界框信息
    line = annotation_line.split()
    # 打开图像文件
    image = Image.open(line[0])
    # 获取原始图像尺寸
    iw, ih = image.size
    # 获取输入尺寸
    h, w = input_shape
    # 解析边界框数据
    box = np.array([np.array(list(map(int, box.split(',')))) for box in line[1:]])

    # 计算缩放比例和调整后的图像尺寸
    scale = min(w/iw, h/ih)
    nw = int(iw*scale)
    nh = int(ih*scale)
    # 计算图像在新画布上的位置
    dx = (w-nw)//2
    dy = (h-nh)//2
    image_data = 0

    # 调整图像尺寸
    image = image.resize((nw, nh), Image.BICUBIC)
    # 创建新的画布并粘贴调整后的图像
    new_image = Image.new('RGB', (w, h), (128, 128, 128))
    new_image.paste(image, (dx, dy))
    # 将图像转换为模型输入格式
    image_data = np.array(new_image)/255.

    # 初始化用于训练的边界框数据
    box_data = np.zeros((max_boxes, 5))
    # 调整边界框坐标
    if len(box) > 0:
        np.random.shuffle(box)
        if len(box) > max_boxes: box = box[:max_boxes]
        box[:, [0, 2]] = box[:, [0, 2]] * scale + dx
        box[:, [1, 3]] = box[:, [1, 3]] * scale + dy
        box_data[:len(box)] = box

    # 返回处理后的图像数据和边界框数据
    return image_data, box_data

In [12]:
def preprocess_true_boxes(true_boxes, input_shape, anchors, num_classes):
    '''Preprocess true boxes to training input format'''

    # 检查所有边界框的类别ID是否小于类别总数
    assert (true_boxes[..., 4] < num_classes).all(), 'class id must be less than num_classes'
    
    # 默认设置为3层锚点
    num_layers = len(anchors) // 3  
    # 设置每层使用的锚点
    anchor_mask = [[6, 7, 8], [3, 4, 5], [0, 1, 2]] if num_layers == 3 else [[3, 4, 5], [1, 2, 3]]

    # 将真实边界框数据和输入尺寸转换为适当的格式
    true_boxes = np.array(true_boxes, dtype='float32')
    input_shape = np.array(input_shape, dtype='int32')
    
    # 计算边界框中心点和宽高
    boxes_xy = (true_boxes[..., 0:2] + true_boxes[..., 2:4]) // 2
    boxes_wh = true_boxes[..., 2:4] - true_boxes[..., 0:2]
    # 将边界框坐标转换为相对于输入尺寸的比例
    true_boxes[..., 0:2] = boxes_xy / input_shape[::-1]
    true_boxes[..., 2:4] = boxes_wh / input_shape[::-1]

    # 初始化 y_true
    m = true_boxes.shape[0]
    grid_shapes = [input_shape // {0: 32, 1: 16, 2: 8}[l] for l in range(num_layers)]
    y_true = [np.zeros((m, grid_shapes[l][0], grid_shapes[l][1], len(anchor_mask[l]), 5 + num_classes),dtype='float32') for l in range(num_layers)]

    # 扩展锚点的维度以适应广播
    anchors = np.expand_dims(anchors, 0)
    anchor_maxes = anchors / 2.
    anchor_mins = -anchor_maxes
    valid_mask = boxes_wh[..., 0] > 0

    for b in range(m):
        # 排除无效的行
        wh = boxes_wh[b, valid_mask[b]]
        if len(wh) == 0: continue
        # 扩展维度以适应广播
        wh = np.expand_dims(wh, -2)
        box_maxes = wh / 2.
        box_mins = -box_maxes

        intersect_mins = np.maximum(box_mins, anchor_mins)
        intersect_maxes = np.minimum(box_maxes, anchor_maxes)
        intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)
        intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
        box_area = wh[..., 0] * wh[..., 1]
        anchor_area = anchors[..., 0] * anchors[..., 1]
        iou = intersect_area / (box_area + anchor_area - intersect_area)

        # 为每个真实边界框找到最匹配的锚点
        best_anchor = np.argmax(iou, axis=-1)

        for t, n in enumerate(best_anchor):
            for l in range(num_layers):
                if n in anchor_mask[l]:
                    i = np.floor(true_boxes[b, t, 0] * grid_shapes[l][1]).astype('int32')
                    j = np.floor(true_boxes[b, t, 1] * grid_shapes[l][0]).astype('int32')
                    k = anchor_mask[l].index(n)
                    c = true_boxes[b, t, 4].astype('int32')
                    # 填充 y_true 张量
                    y_true[l][b, j, i, k, 0:4] = true_boxes[b, t, 0:4]
                    y_true[l][b, j, i, k, 4] = 1
                    y_true[l][b, j, i, k, 5 + c] = 1

    return y_true

In [13]:
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator
    - annotation_lines: 图像的注释行列表，每行包含图像路径和边界框信息
    - batch_size: 批次大小
    - input_shape: 模型的输入尺寸
    - anchors: YOLO模型使用的锚点
    - num_classes: 目标类别的总数
    '''

    # 获取注释行的数量
    n = len(annotation_lines)
    # 初始化索引
    i = 0

    # 无限循环，持续生成数据
    while True:
        # 初始化批次内的图像和边界框列表
        image_data = []
        box_data = []

        # 构建一个批次的数据
        for b in range(batch_size):
            # 如果索引为0，则随机打乱注释行
            if i == 0:
                np.random.shuffle(annotation_lines)

            # 获取单个图像及其边界框数据
            image, box = get_random_data(annotation_lines[i], input_shape, random=False)
            # 添加到批次数据列表
            image_data.append(image)
            box_data.append(box)

            # 更新索引，循环使用注释行
            i = (i + 1) % n

        # 将图像和边界框数据列表转换为NumPy数组
        image_data = np.array(image_data)
        box_data = np.array(box_data)

        # 预处理边界框数据
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)

        # 产出图像数据、标签数据和占位符
        yield [image_data, *y_true], np.zeros(batch_size)

In [14]:
# anchors = np.array([[10,13], [16,30], [33,23], [30,61], [62,45], [59,119],[116,90], [156,198], [373,326]])
# 定义YOLOv3模型的锚点
anchors = np.array([[25,39], [38,91], [62,51], [71,136], [123,214], [127,95], [219,293], [250,148], [394,298]])

# 定义要检测的类别
class_names = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
               'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
num_classes = len(class_names)  # 类别的数量

input_shape = (416, 416)  # 输入尺寸，必须是32的倍数

# 创建YOLOv3模型
model = create_model(input_shape, anchors, num_classes, freeze_body=2, weights_path='yolov3.h5')

# 设置TensorBoard日志
logging = TensorBoard(log_dir='logs/')

# 设置模型保存的回调函数
checkpoint = ModelCheckpoint('ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
                             monitor='val_loss', save_weights_only=True, save_best_only=True, save_freq="epoch")

# 设置学习率衰减的回调函数
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

# 读取训练数据
annotation_path = '2007_trainval.txt'
val_split = 0.1  # 验证集的比例
with open(annotation_path) as f:
    lines = f.readlines()

# 分割训练集和验证集
num_val = int(len(lines) * val_split)
num_train = len(lines) - num_val

# 编译模型和设置损失函数
model.compile(optimizer=Adam(lr=1e-3), loss={'yolo_loss': lambda y_true, y_pred: y_pred})

# 打印训练和验证样本数量
batch_size = 8
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))

# 开始训练模型
model.fit(x=data_generator(lines[:num_train], batch_size, input_shape, anchors, num_classes),
          steps_per_epoch=max(1, num_train // batch_size),
          validation_data=data_generator(lines[num_train:], batch_size, input_shape, anchors, num_classes),
          validation_steps=max(1, num_val // batch_size),
          epochs=25,
          initial_epoch=0,
          callbacks=[logging, checkpoint])  # reduce_lr为可选项

# 保存最终的模型
# model.save('final.h5')

Create YOLOv3 model with 9 anchors and 20 classes.
Load weights yolov3.h5.
Freeze the first 249 layers of total 252 layers.
Train on 4059 samples, val on 450 samples, with batch size 8.
Epoch 1/25
507/507 [==============================] - 59s 99ms/step - loss: 686.8931 - val_loss: 39.7856
Epoch 2/25
507/507 [==============================] - 48s 94ms/step - loss: 36.8385 - val_loss: 31.2308
Epoch 3/25
507/507 [==============================] - 47s 93ms/step - loss: 30.4225 - val_loss: 27.2752
Epoch 4/25
507/507 [==============================] - 47s 92ms/step - loss: 27.4803 - val_loss: 26.3845
Epoch 5/25
507/507 [==============================] - 47s 92ms/step - loss: 25.7425 - val_loss: 24.7107
Epoch 6/25
507/507 [==============================] - 47s 92ms/step - loss: 24.5535 - val_loss: 23.1658
Epoch 7/25
507/507 [==============================] - 47s 93ms/step - loss: 23.6432 - val_loss: 23.0214
Epoch 8/25
507/507 [==============================] - 47s 92ms/step - loss: 23.2667 -

In [15]:
model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) 

for i in range(len(model.layers)):
    model.layers[i].trainable = True

batch_size = 8
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit(  x=data_generator(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train // batch_size),
            validation_data=data_generator(lines[num_train:], batch_size, input_shape, anchors,num_classes),
            validation_steps=max(1, num_val // batch_size),
            epochs=50,
            initial_epoch=25,
            callbacks=[logging, checkpoint,reduce_lr]) #reduce_lr #_generator

model.save('final0219.h5')

Train on 4059 samples, val on 450 samples, with batch size 8.
Epoch 26/50
507/507 [==============================] - 55s 97ms/step - loss: 17.4452 - val_loss: 18.0984
Epoch 27/50
507/507 [==============================] - 46s 91ms/step - loss: 16.9324 - val_loss: 18.0666
Epoch 28/50
507/507 [==============================] - 46s 91ms/step - loss: 16.9438 - val_loss: 18.2090
Epoch 29/50
507/507 [==============================] - 46s 91ms/step - loss: 16.9735 - val_loss: 17.9186
Epoch 30/50
507/507 [==============================] - 46s 91ms/step - loss: 16.9865 - val_loss: 17.8229
Epoch 31/50
507/507 [==============================] - 46s 91ms/step - loss: 16.7233 - val_loss: 18.0692
Epoch 32/50
507/507 [==============================] - 46s 90ms/step - loss: 17.0337 - val_loss: 17.9939
Epoch 33/50
507/507 [==============================] - 46s 91ms/step - loss: 16.9882 - val_loss: 17.9747

Epoch 00033: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 34/50
507/5